In [ ]:
import os
import csv
import pandas as pd
from Bio import Entrez
from tqdm import tqdm
from Bio import SeqIO

### Setup

Primeiramente é feita a filtragem dos Dados iniciais.

In [ ]:
file_paths = os.listdir('./data/')
file_paths = list(set(file_paths) - set(['data.csv', 'sequences']))
print(file_paths)

In [ ]:
data = []

for path in file_paths:
    with open('./data/' + path, 'r') as f:
        reader = csv.reader(f, delimiter='\t')

        class_name = path.split('.')[0].split('_')[1]

        for row in reader:
            if (row[6] == '+'):
                data.append([class_name] + row)

In [ ]:
header = ['Class', 'Chr', 'Source Annotation', 'Class/Order/Superfamily', 'Start', 'End', 'Score', 'Strand', 'Phase', 'Attributes']

data_df = pd.DataFrame(data, columns=header)
data_df = data_df[['Class', 'Chr', 'Start', 'End']]

data_df.head()

In [ ]:
data_df.to_csv('./data/data.csv')

In [ ]:
## Checando os possíveis valores dos cromossomos
data_df.Chr.unique()

Em sequência será feita a extração dos genomas por meio de Biopython

In [ ]:
id_dict = {"LR618874.1": "Chr_1.txt", "LR618875.1": "Chr_2.txt", "LR618876.1": "Chr_3.txt", "LR618877.1": "Chr_4.txt", 
           "LR618878.1": "Chr_5.txt", "LR618879.1": "Chr_6.txt", "LR618880.1": "Chr_7.txt", "LR618881.1": "Chr_8.txt", 
           "LR618882.1": "Chr_9.txt", "LR618883.1": "Chr_10.txt", "AY506529.1":"Chr_Mt.txt", "X86563.2": "Chr_Pt.txt"}

Entrez.email = "pedro.guilherme2305@usp.br"
for id in tqdm(id_dict, total=len(id_dict)):
    stream = Entrez.efetch(db="nuccore", id=id, rettype="fasta")
    
    with open(f"./data/sequences/{id_dict[id]}", "w") as file:
        file.write(stream.read())
    stream.close()

### Processamento

In [ ]:
data_df = pd.read_csv("./data/data.csv")
chromosomes_to_keep = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', 'Mt', 'Pt']
data_df = data_df.query("Chr in @chromosomes_to_keep") 

In [ ]:
aux_list = []

for chromosome in chromosomes_to_keep:
    rows = data_df.query(f"Chr == '{chromosome}'").to_dict(orient="records")
    record = SeqIO.read(f"./data/sequences/Chr_{chromosome}.txt", "fasta")
    for row in tqdm(rows, total=len(rows)):
        aux_dict = dict()

        aux_dict['Chr'] = row['Chr']
        aux_dict['Sequence'] = record[row['Start']:row['End']].seq
        aux_dict['Class'] = row['Class']

        aux_list.append(aux_dict)


In [ ]:
final_df = pd.DataFrame(aux_list)
final_df

In [ ]:
final_df.to_csv('./data/final.csv')